In [ ]:
### Imports
### I used google colab to run the code so you will see many references to it, you can just ignore the colab references if you use another IDE

import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### Create DataFrame

df = pd.read_excel('PATH TO YOUR FILE')

In [ ]:
# Calculate the porcentage of missiong data
# You should repeat this process for all columns that contain relevant data
# I had daily mean, max and minimum temperature columns as well as mean daily humidity, so i did this 4 times
# World Meteorological Organization recomends no more han 20% missing data
# However you shold consider the missing data pattern
# If you have data missing for long consecutive periods, you shold consider another method for filling in missing data because the one showed below is fragile in those scenarios
print('Percent missing data for Min Temp:')
pct_missing = df['tempMin'].isna().mean() * 100
print(round(pct_missing,2))
print('Percent missing data for Mean Temp:')
pct_missing = df['tempMed'].isna().mean() * 100
print(round(pct_missing,2))
print('Percent missing data for Max Temp:')
pct_missing = df['tempMax'].isna().mean() * 100
print(round(pct_missing,2))
print('Percent missing data for humidity:')
pct_missing = df['HumidadeMed'].isna().mean() * 100
print(round(pct_missing,2))

Percent missing data for Min Temp:
0.39
Percent missing data for Mean Temp:
0.73
Percent missing data for Max Temp:
0.36
Percent missing data for humidity:
10.42


In [ ]:
# Filter the data by year to calculate your T95
# you may adjust this period to one that is relevant to your data, but it whould be at least 30 years
startT95 = pd.to_datetime('1981-01-01')
endT95 = pd.to_datetime('2010-12-31')
# Define T95 based on the filter
dfT95 = df[(df['date'] >= startT95) & (df['date'] <= endT95)]
# Calculate T95
T95 = dfT95['tempMean'].quantile(0.95)

In [ ]:
# Make sure your data is ordered and filled
startDate = df['date'].min()
endDate = df['date'].max()
complete_index = pd.date_range(start=startDate, end=endDate)
df = df.set_index('date').reindex(complete_index).reset_index()
df['tempMean'] = df['tempMean'].fillna(method='ffill').fillna(method='bfill')
df['tempMax'] = df['tempMax'].fillna(method='ffill').fillna(method='bfill')
df['tempMin'] = df['tempMin'].fillna(method='ffill').fillna(method='bfill')
df['HumidadeMed'] = df['HumidadeMed'].fillna(method='ffill').fillna(method='bfill')
# Calculate the average temperature for the 3 day period and the 30 day period
df['TDP'] = df['tempMean'].shift(-2).rolling(window=3, min_periods=1).mean()
df['30DP'] = df['tempMean'].rolling(window=30, min_periods=1).mean()
# Keep in mind you should not use the 30 first days of your dataframe due to the calculation of the 30 day period
# Calculate EHIaccl
df['EHIaccl'] = df['TDP'] - df['30DP']
# Calculate EHIsigg
df['EHIsigg'] = df['TDP'] - T95
# Calculate EHF
df['EHF'] =  np.where(df['EHIaccl'] > 1, df['EHIsigg'] * df['EHIaccl'], df['EHIsigg'] * 1)
# Define the Heat Wave days
df['isHW'] = False
for i in range(len(df)):
    if df.loc[i, 'EHF'] > 0:
        df.loc[i:i+2, 'isHW'] = True

<ipython-input-5-a6d85a1717d8>:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['tempMed'] = df['tempMed'].fillna(method='ffill').fillna(method='bfill')
<ipython-input-5-a6d85a1717d8>:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['tempMax'] = df['tempMax'].fillna(method='ffill').fillna(method='bfill')
<ipython-input-5-a6d85a1717d8>:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['tempMin'] = df['tempMin'].fillna(method='ffill').fillna(method='bfill')
<ipython-input-5-a6d85a1717d8>:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['HumidadeMed'] = df['HumidadeMed'].fillna(method='ffill').fillna(method='bfill')


In [ ]:
# Caculate EHF85
positiveEHFs = df[df['EHF'] > 0]
EHF85 = positiveEHFs['EHF'].quantile(0.85)
EHF85_3x = EHF85 * 3

In [ ]:
# Define the conditions and name possible heat wave intensities based on the EHF value
conditions = [
    (df['EHF'] <= 0),
    (df['EHF'] > 0) & (df['EHF'] < EHF85),
    (df['EHF'] >= EHF85) & (df['EHF'] < EHF85_3x),
    (df['EHF'] >= EHF85_3x)
]
intensities = ['Not HW','Low-Intensity','Severe', 'Extreme']
# Define the heat wave days's intensities
df['HWDay_Intensity'] = np.select(conditions, intensities)

# Calculate daily thermal range
df['thermalRange'] = df['tempMax'] - df['tempMin']

# Create a 'year' column to facilitate future calculations
df['year'] = df['index'].dt.year

In [ ]:
# Calculate the number of heat wave days per year
HWDays = df.groupby('year')['isHW'].sum()
print(HWDays)

year
1981      0
1982      0
1983      0
1984      0
1985      0
1986      0
1987      8
1988      0
1989      0
1990      0
1991      0
1992      0
1993      0
1994      0
1995      5
1996      0
1997     28
1998     95
1999     11
2000      5
2001     14
2002      3
2003     21
2004     12
2005     59
2006     49
2007      7
2008     79
2009     92
2010    157
2011     92
2012     38
2013     32
2014     49
2015    116
2016    161
2017     82
2018     50
2019     38
2020     91
2021     35
2022     69
2023    161
2024      0
Name: isHW, dtype: int64


In [ ]:
# Define a function to count the number of heat waves per year
def count_HW_periods(df):
    df['group'] = (df['isHW'] != df['isHW'].shift()).cumsum()
    HW_periods = df[df['isHW']].groupby(['year', 'group']).size().reset_index(name='count')
    HW_periods = HW_periods[HW_periods['count'] >= 3]
    result = HW_periods.groupby('year').size().reset_index(name='num_HW_periods')
    return result

# Count the number of heat waves per year
result = count_HW_periods(df)
print(result)

    year  num_HW_periods
0   1987               2
1   1995               1
2   1997               4
3   1998              14
4   1999               3
5   2000               1
6   2001               1
7   2002               1
8   2003               6
9   2004               2
10  2005              10
11  2006               8
12  2007               2
13  2008              12
14  2009              10
15  2010              18
16  2011              12
17  2012               7
18  2013               8
19  2014               5
20  2015              10
21  2016              22
22  2017              12
23  2018               9
24  2019               7
25  2020              14
26  2021               8
27  2022              12
28  2023              13


In [ ]:
# Define a function to calculate the mean heat wave duration per year
def calculate_average_duration(df):
    # Identify the groups of heat wave periods
    df['group'] = (df['isHW'] != df['isHW'].shift()).cumsum()

    # Filter the heat wave periods
    HW_periods = df[df['isHW']].groupby(['year', 'group']).size().reset_index(name='count')

    # Filter only the periods with at least 3 days
    HW_periods = HW_periods[HW_periods['count'] >= 3]

    # Calculate the average heat wave duration per year
    average_duration = HW_periods.groupby('year')['count'].mean().reset_index(name='average_duration')

    return average_duration

# Print the average heat wave duration per year
average_duration = calculate_average_duration(df)
print(average_duration)

    year  average_duration
0   1987          4.000000
1   1995          5.000000
2   1997          7.000000
3   1998          6.785714
4   1999          3.666667
5   2000          5.000000
6   2001         14.000000
7   2002          3.000000
8   2003          3.500000
9   2004          6.000000
10  2005          5.900000
11  2006          6.125000
12  2007          3.500000
13  2008          6.583333
14  2009          9.200000
15  2010          8.722222
16  2011          7.666667
17  2012          5.428571
18  2013          4.000000
19  2014          9.800000
20  2015         11.600000
21  2016          7.318182
22  2017          6.833333
23  2018          5.555556
24  2019          5.428571
25  2020          6.500000
26  2021          4.375000
27  2022          5.750000
28  2023         12.384615


In [ ]:
# Create new column for the mean thermal range (MTR) and median thermal range (MedTR) and assing initial value
df['MTR'] = 0
df['MedTR'] = 0

# Identify periods with consecutive true values for isHW
in_period = False
start_idx = 0

for i in range(len(df)):
    if df.loc[i, 'isHW']:
        if not in_period:
            in_period = True
            start_idx = i
    else:
        if in_period:
            in_period = False
            # Calculate the mean and median for the daily thermal range values in each heat wave
            end_idx = i
            mean_thermalRange = df.loc[start_idx:end_idx-1, 'thermalRange'].mean()
            median_thermalRange = df.loc[start_idx:end_idx-1, 'thermalRange'].median()
            # Fill in the MTR column with the calculated mean
            df.loc[start_idx:end_idx-1, 'MTR'] = mean_thermalRange
            df.loc[start_idx:end_idx-1, 'MedTR'] = median_thermalRange

# Treat the last period in case it is the last dataframe entry
if in_period:
    mean_thermalRange = df.loc[start_idx:, 'thermalRange'].mean()
    df.loc[start_idx:, 'MTR'] = mean_thermalRange

<ipython-input-11-40044a4ca87d>:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.875000000000002' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[start_idx:end_idx-1, 'MTR'] = mean_thermalRange
<ipython-input-11-40044a4ca87d>:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.800000000000002' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[start_idx:end_idx-1, 'MedTR'] = median_thermalRange


In [ ]:
# Create a function to calculate number of heatwaves per month across the dataframe
# This is intended to find out which are the months that heat waves happen most commonly
def count_heatWaves_by_month(df):
    # Identify groups
    df['group'] = (df['isHW'] != df['isHW'].shift()).cumsum()

    # Filter the heat waves
    Heat_Waves = df[df['isHW']].groupby(['group']).agg(
        start_date=('index', 'first'),
        end_date=('index', 'last')
    ).reset_index()

    # Extract the month of the start of the heat wave
    Heat_Waves['month'] = Heat_Waves['start_date'].dt.month

    # Count the number of heat waves per month across the years
    HW_by_month = Heat_Waves.groupby(['month']).size().reset_index(name='num_periods')

    return HW_by_month

# Executar a função
HW_by_month = count_heatWaves_by_month(df)
print(HW_by_month)

    month  num_periods
0       1            7
1       2            5
2       3            3
3       4            7
4       5           20
5       6           14
6       7           20
7       8           30
8       9           29
9      10           44
10     11           33
11     12           22


In [ ]:
# Create e homogenous column to define the intensity of each heat wave instead of each heat wave day

# Auxilery function to determine the most frequent value in the HWDay_Intensity
def definir_hw_intensity(grupo):
    # Check the most frequent value in HWDay_Intensity
    most_frequent_value = grupo['HWDay_Intensity'].mode()[0]

    # Apply rules to define the HW Intensity
    if most_frequent_value in ['Not HW', 'Low-Intensity']:
        return 'Low Intensity'
    elif most_frequent_value == 'Severe':
        return 'Severe'
    elif most_frequent_value == 'Extreme':
        return 'Extreme'

# Create column with 'Not HW' for all days
df['HW_Intensity'] = 'Not HW'

# Identify consecutive True values in isHW
df['bloco'] = (df['isHW'] != df['isHW'].shift()).cumsum()

# Filter only the blocks where isHW is True (which are the heat wave periods)
blocos_hw = df[df['isHW'] == True].groupby('bloco')

# Apply the function in each block to define the value for HW_Intensity
for nome_bloco, grupo in blocos_hw:
    valor_intensidade = definir_hw_intensity(grupo)
    df.loc[df['bloco'] == nome_bloco, 'HW_Intensity'] = valor_intensidade

# Remove the temporary column
df.drop(columns=['bloco'], inplace=True)

In [ ]:
# create new 'days_since_HW' column
df['days_since_HW'] = 0

# Variable to count days since last 'True' value in isHW
dias_desde_ultima_hw = 0

# Variable to save the number of days since last heat wave
valor_atual_hw = 0

# Iterate over the DataFrame
for i in range(len(df)):
    if df['isHW'].iloc[i] == True:
        # If it is the first day of the new heat wave, define the block value with the current count
        if df['isHW'].iloc[i-1] == False or i == 0:
            valor_atual_hw = dias_desde_ultima_hw

        # Atribute the same value for all heat wave days
        df['days_since_HW'].iloc[i] = valor_atual_hw

        # Reset count
        dias_desde_ultima_hw = 0
    else:
        # Increment the count
        dias_desde_ultima_hw += 1
        # Atribute 0 to the days between heat waves
        df['days_since_HW'].iloc[i] = 0

A saída de streaming foi truncada nas últimas 5000 linhas.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['days_since_HW'].iloc[i] = 0
<ipython-input-14-fbac7f40fc33>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_since_HW'].iloc[i] = 0
<ipython-input-14-fbac7f40fc33>:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when yo

In [ ]:
# Create new column for the mean humidity and assing initial value
df['Mean_HW_Humidity'] = 0

# Identify periods with consecutive true values for isHW
in_period = False
start_idx = 0

for i in range(len(df)):
    if df.loc[i, 'isHW']:
        if not in_period:
            in_period = True
            start_idx = i
    else:
        if in_period:
            in_period = False
            # Calculate the mean humidity in each heat wave
            end_idx = i
            mean_humidity = df.loc[start_idx:end_idx-1, 'HumidadeMed'].mean()
            # Fill in the MeanHumidity column with the calculated mean
            df.loc[start_idx:end_idx-1, 'Mean_HW_Humidity'] = mean_humidity

# Treat the last period in case it is the last dataframe entry
if in_period:
    mean_humidity = df.loc[start_idx:, 'HumidadeMed'].mean()
    df.loc[start_idx:, 'Mean_HW_Humidity'] = mean_humidity

<ipython-input-15-8cf13e7a7706>:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '83.7' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[start_idx:end_idx-1, 'Mean_HW_Humidity'] = mean_humidity


In [ ]:
# Create new column 'HW_duration' with placeholder value
df['HW_duration'] = 0

# Identify blocks of consecutive True valuers in isHW
df['bloco'] = (df['isHW'] != df['isHW'].shift()).cumsum()

# Filter only the blocks where isHW is True (the heat wave periods)
blocos_hw = df[df['isHW'] == True].groupby('bloco')

# calculate the duration and atribute it to the HW_duration for each heat wave block
for nome_bloco, grupo in blocos_hw:
    duracao = len(grupo)  # Calculate HW duration
    df.loc[df['bloco'] == nome_bloco, 'HW_duration'] = duracao

# Remove temporary column
df.drop(columns=['bloco'], inplace=True)

In [ ]:
# Filter the DataFrame for the 1981 to 2010 period
df_referencia = df[(df['index'].dt.year >= 1981) & (df['index'].dt.year <= 2010)]

# Create a column for the month and day to group the data
df_referencia['mes_dia'] = df_referencia['index'].dt.strftime('%m-%d')

# Calculate the mean maximum temperature for each day during the selected period
media_tempMax = df_referencia.groupby('mes_dia')['tempMax'].mean()

# Create a reference column in the original DataFrame
df['mes_dia'] = df['index'].dt.strftime('%m-%d')

# Calculate maximum temperature anomaly
df['tempAnom'] = df['tempMax'] - df['mes_dia'].map(media_tempMax)

# Drop the temporary column
df.drop(columns=['mes_dia'], inplace=True)

# Create column for the HeatWave mean temperature anomaly with placeholder value
df['mean_temp_anom'] = 0

# Identify the heatwaves
df['bloco'] = (df['isHW'] != df['isHW'].shift()).cumsum()

# Filter the heatwave blocks
blocos_hw = df[df['isHW'] == True].groupby('bloco')

# Calculate mean temperature anomaly with the maximum temperatures for the heatwaves
for nome_bloco, grupo in blocos_hw:
    # Calculate the mean anomalies from 'tempAnom'
    media_anomalia = grupo['tempAnom'].mean()
    # Assign the calculated mean to the 'mean_temp_anom' for the heatwave days
    df.loc[df['bloco'] == nome_bloco, 'mean_temp_anom'] = media_anomalia

# Drop the temporary column
df.drop(columns=['bloco'], inplace=True)

<ipython-input-17-53cf94da04e4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_referencia['mes_dia'] = df_referencia['index'].dt.strftime('%m-%d')
<ipython-input-17-53cf94da04e4>:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.4250000000000025' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['bloco'] == nome_bloco, 'mean_temp_anom'] = media_anomalia


In [ ]:
### Export table
df.to_excel('YOUR_TABLE_NAME.xlsx', index=False)
### Download table
from google.colab import files
files.download('YOUR_TABLE_NAME.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>